In [ ]:
!pip install elevenlabs
!pip install openai

In [27]:
import requests
import json
import elevenlabs
from openai import OpenAI
import time

In [29]:
with open('../credentials.json', 'r') as file:
    cred = json.load(file)

elevenlabs.set_api_key(cred["elevenlabs_api_key"])
client = OpenAI(api_key=cred["openai_api_key"])

assistant_id = "asst_nv9sFR2LXHefrpwws4EUT6xU"
voice_id = "vDxT1hhO5CjgzUI7e2vF"

my_assistant = client.beta.assistants.retrieve(assistant_id)

new_thread = client.beta.threads.create()

In [ ]:
def video_generation(audio_path):
    import subprocess

# Define the command and arguments
command = [
    'python', './SadTalker/inference.py',
    '--driven_audio', audio_path,
    '--source_image', './SadTalker/example_img.jpg',
    '--result_dir', './results'
]

# Run the command
process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the process was successful
if process.returncode == 0:
    print("Command executed successfully")
    print(process.stdout)
else:
    print("Command failed")
    print(process.stderr)

In [30]:
def new_question(question_text):
  message_request = client.beta.threads.messages.create(
      thread_id = new_thread.id,
      role = "user",
      content = question_text
  )

  run = client.beta.threads.runs.create(
    thread_id = message_request.thread_id,
    assistant_id = my_assistant.id
  )

  run = client.beta.threads.runs.retrieve(
    thread_id = message_request.thread_id,
    run_id = run.id
  )

  def wait_for_completed():
      while True:
          response = client.beta.threads.runs.retrieve(thread_id = new_thread.id, run_id = run.id)
          if response.status == "completed":
              return response
          time.sleep(1)

  run_return = wait_for_completed()

  messages = client.beta.threads.messages.list(
    thread_id = message_request.thread_id
  )

  audio = elevenlabs.generate(
      text = messages.data[0].content[0].text.value,
          voice = voice_id
  )

  elevenlabs.save(audio, "test.wav")
  video_generation("test.wav")

  #elevenlabs.play(audio)
  

In [1]:
!python --version

Python 3.8.0


In [31]:
new_question("How can I relax?")

In [ ]:
!python -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!python -m pip install moviepy
!python -m pip install imageio==2.4.1
!python -m pip install -r requirements.txt
!python -m pip install cmake==3.25.2
!python -m pip install boost==0.1
!python -m pip install dlib-bin
!python -m pip install basicsr==1.4.2
!python -m pip install facexlib==0.2.5
!python -m pip install kornia==0.6.10
!python -m pip install face-alignment==1.3.5 yacs==0.1.8
### install gpfgan for enhancer
!python -m pip install git+https://github.com/TencentARC/GFPGAN

In [3]:
import glob
import matplotlib.pyplot as plt
from PIL import Image
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

In [5]:
pip install face-alignment==1.3.4

  Using cached face_alignment-1.3.4-py2.py3-none-any.whl
  Attempting uninstall: face-alignment
    Found existing installation: face-alignment 1.3.5
    Uninstalling face-alignment-1.3.5:
      Successfully uninstalled face-alignment-1.3.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python ./SadTalker/inference.py \
  --driven_audio ./SadTalker/examples/driven_audio/RD_Radio31_000.wav \
  --source_image ./SadTalker/example_img.jpg \
  --result_dir ./results

In [17]:
import os

# Path to the directory you want to scan
directory_path = 'results/2023_12_19_20.56.50/'

# List to hold the names of folders
file_names = []

# Walk through the directory
for entry in os.scandir(directory_path):
    
    file_names.append(entry.name)

# Print the folder names
print(file_names)

['example_img##RD_Radio31_000.mat', 'example_img##RD_Radio31_000.mp4', 'example_img##RD_Radio31_000.txt', 'first_frame_dir']


In [22]:
import os
print(os.getcwd())
%ls
directory_path = 'results/2023_12_19_20.56.50/'
file_name = 'example_img##RD_Radio31_000.mp4'

full_path = os.path.join(directory_path, file_name)

os.startfile(full_path)
'''
if os.path.exists(video_path):
    print("File exists")
    os.startfile(video_path)
else:
    print("File not found")
    '''

c:\Users\jonathancallahan\Projects\guidance
 Volume in drive C is Windows
 Volume Serial Number is C402-5B7C

 Directory of c:\Users\jonathancallahan\Projects\guidance

12/19/2023  09:19 PM    <DIR>          .
12/19/2023  09:19 PM    <DIR>          ..
11/17/2023  07:21 PM    <DIR>          .ipynb_checkpoints
11/26/2023  02:13 PM    <DIR>          .venv
12/19/2023  07:31 PM    <DIR>          checkpoints
11/26/2023  12:30 PM    <DIR>          content
11/17/2023  06:13 PM                10 README.md
12/19/2023  08:56 PM    <DIR>          results
12/19/2023  07:31 PM    <DIR>          SadTalker
12/19/2023  09:19 PM            10,914 text_to_speech.ipynb
11/17/2023  11:05 PM    <DIR>          video
               2 File(s)         10,924 bytes
               9 Dir(s)  522,661,634,048 bytes free


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'results/2023_12_19_20.56.50/example_img##RD_Radio31_000.mp4'